In [1]:
#fundamental tools for the game environment model
import numpy as np
import math
import random
import itertools

# for visualations
import matplotlib
import matplotlib.pyplot as plt

In [2]:
import exploreModel
import explore_Viewer

pygame 2.5.1 (SDL 2.28.2, Python 3.11.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
import multiprocess
from launch_async_viewer import launch_viewer

In [4]:
env = exploreModel.exploreGame()
qq = multiprocess.Queue()

In [5]:
emap = env.generate_frame()
hmap = env.get_trace()

In [6]:
qq.put(emap)
qq.put(hmap)

In [7]:
p = multiprocess.Process(target=launch_viewer, args=[qq])

In [8]:
p.start()

In [9]:
for k in range(10000*1000):
    env.update(np.random.choice(5))
    emap = env.generate_frame()
    hmap = env.get_trace()

    if qq.empty():
        qq.put(emap)
        qq.put(hmap)